In [1]:
import sys

from typing import Optional
from pathlib import Path

from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import RedirectResponse

# Add persyn root to sys.path
sys.path.insert(0, '/home/rob/persyn/')
sys.path.insert(0, '/home/rob/persyn/interaction')

from interaction.interact import Interact

# Color logging
# from utils.color_logging import log


import os
os.environ['PERSYN_CONFIG'] = '/home/rob/persyn/config/anna.yaml'

# Bot config
from utils.config import load_config

interact = Interact(load_config())

/home/rob/persyn/interaction/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rob/persyn/interaction/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rob/persyn/interaction/env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.1 when using version 1.0.2. This

2022-11-27 16:09:23,308 loading file /home/rob/.flair/models/sentiment-en-mix-distillbert_4.pt


POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.290s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.028s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-entities-v0/_search [status:200 duration:0.022s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-relationships-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-opinions-v0/_search [status:200 duration:0.028s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-beliefs-v0/_search [status:200 duration:0.028s]


In [2]:
service='https://unit-16.slack.com/'
channel='D02NK563878'

In [47]:
convo_ids = interact.recall.ltm.es.search( # pylint: disable=unexpected-keyword-arg
    _source=['convo_id'],
    aggs={
        "patterns" : {
            "terms" : {
                "field" : "convo_id.keyword",
                "size": 10000
            }
        }
    },
    index='anna-summaries-v0',
    query={"match_all": {}},
    size=10
)['aggregations']


POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.069s]


In [50]:
all_convo_ids = [v['key'] for v in convo_ids['patterns']['buckets']]

In [82]:
for convo_id in all_convo_ids:
    summary = interact.recall.ltm.es.search(
        index='anna-summaries-v0',
        query={
            "bool":{
                "must": [
                    {"match": {"convo_id": { "query": convo_id}}}
                ]
            }
        }
    )

    for doc in summary.body['hits']['hits']:
        convo = interact.recall.ltm.es.search( # pylint: disable=unexpected-keyword-arg
            index='anna-conversations-v0',
            _source=["convo_id"],
            aggs={
                "patterns" : {
                    "terms" : {
                        "field" : "entity_id.keyword",
                        "size": 100
                    }
                }
            },
            query={
                "bool": {
                    "must": [
                        {"match": {"convo_id": {"query": convo_id}}}
                    ]
                }
            },
            size=10
        )['aggregations']

        speakers = [v['key'] for v in convo['patterns']['buckets']]
        
        source = doc['_source']
        source['speaker_ids'] = speakers

        reply = interact.recall.ltm.es.update(
            index="anna-summaries-v0",
            id=doc['_id'],
            doc=source
        )
        print(reply)

POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.022s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/rmM6on4BSsm-lZhBqgs1 [status:200 duration:0.028s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/r2M7on4BSsm-lZhBwAu6 [status:200 duration:0.028s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.023s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/sWM9on4BSsm-lZhBBAsX [status:200 duration:0.030s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.023s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/v2N5o

{'_index': 'anna-summaries-v0', '_id': 'rmM6on4BSsm-lZhBqgs1', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1629, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'r2M7on4BSsm-lZhBwAu6', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1630, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'sWM9on4BSsm-lZhBBAsX', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1631, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'v2N5on4BSsm-lZhBbQuS', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1632, '_primary_term': 17}


POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/wGN6on4BSsm-lZhBXAs5 [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.024s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/wmN-on4BSsm-lZhBCgtb [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.023s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/TmP8nX4BSsm-lZhBNAo8 [status:200 duration:0.022s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.021s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/NWPJoX4BSsm-lZhBYwuL [status:200 duration:0.030s]


{'_index': 'anna-summaries-v0', '_id': 'wGN6on4BSsm-lZhBXAs5', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1633, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'wmN-on4BSsm-lZhBCgtb', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1634, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'TmP8nX4BSsm-lZhBNAo8', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1635, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'NWPJoX4BSsm-lZhBYwuL', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1636, '_primary_term': 17}


POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.024s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/T2P8nX4BSsm-lZhB5Apk [status:200 duration:0.030s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.024s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/UGP_nX4BSsm-lZhBeApN [status:200 duration:0.030s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.023s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/HGbig34Byo-dF4zk_4QF [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.030s]


...

{'_index': 'anna-summaries-v0', '_id': '3Bhdu4AB1O6hlTWgwXOH', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2790, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'zSM6coMBy5mPdc6P1HBW', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2791, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'UOAvd4QB6EXxeIxUAYIP', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2792, '_primary_term': 17}


POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.028s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/ih0yLYIBGYbVpQ0ojaPD [status:200 duration:0.029s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.021s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/Fk0FqoIBRfbdi4KAc09J [status:200 duration:0.021s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.020s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/k6-E9oMBBNi73XUqFaRg [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:2

{'_index': 'anna-summaries-v0', '_id': 'ih0yLYIBGYbVpQ0ojaPD', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2793, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'Fk0FqoIBRfbdi4KAc09J', '_version': 2, 'result': 'noop', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 1624, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'k6-E9oMBBNi73XUqFaRg', '_version': 2, 'result': 'noop', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 1625, '_primary_term': 17}


POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/ra9AAYQBBNi73XUql6bm [status:200 duration:0.023s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.024s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/5a_sEYQBBNi73XUqOKgV [status:200 duration:0.028s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_search [status:200 duration:0.027s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-conversations-v0/_search [status:200 duration:0.026s]
POST https://persyn.es.us-west-2.aws.found.io:443/anna-summaries-v0/_update/o-CWiIQB6EXxeIxUe4Ki [status:200 duration:0.022s]


{'_index': 'anna-summaries-v0', '_id': 'ra9AAYQBBNi73XUql6bm', '_version': 2, 'result': 'noop', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 1626, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': '5a_sEYQBBNi73XUqOKgV', '_version': 2, 'result': 'noop', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 1627, '_primary_term': 17}
{'_index': 'anna-summaries-v0', '_id': 'o-CWiIQB6EXxeIxUe4Ki', '_version': 2, 'result': 'noop', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 1628, '_primary_term': 17}
